In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Endometrioid_Cancer"
cohort = "GSE40785"

# Input paths
in_trait_dir = "../../input/GEO/Endometrioid_Cancer"
in_cohort_dir = "../../input/GEO/Endometrioid_Cancer/GSE40785"

# Output paths
out_data_file = "../../output/preprocess/Endometrioid_Cancer/GSE40785.csv"
out_gene_data_file = "../../output/preprocess/Endometrioid_Cancer/gene_data/GSE40785.csv"
out_clinical_data_file = "../../output/preprocess/Endometrioid_Cancer/clinical_data/GSE40785.csv"
json_path = "../../output/preprocess/Endometrioid_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# Define data availability flags
is_gene_available = True  # Dataset likely contains gene expression data based on background info

# Define which rows in sample characteristics contain our features of interest
trait_row = 1  # The histology information 
age_row = None  # Age data is not available in sample characteristics
gender_row = None  # Gender data is not available in sample characteristics

# Define conversion functions for each variable
def convert_trait(value):
    """
    Convert histology data to a binary indicating Endometrioid_Cancer (1) or not (0).
    """
    if pd.isna(value):
        return None
    
    # Extract the value after the colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary based on histology
    if "Endometrioid" in value:
        return 1  # Presence of Endometrioid cancer
    else:
        return 0  # Other histology types
    
# Since age and gender are not available, we define placeholder functions
def convert_age(value):
    return None

def convert_gender(value):
    return None

# Save metadata for initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(is_final=False, 
                             cohort=cohort, 
                             info_path=json_path, 
                             is_gene_available=is_gene_available, 
                             is_trait_available=is_trait_available)

# If trait data is available, extract clinical features using the sample characteristics dict
if trait_row is not None:
    # Use the sample characteristics dictionary from the previous step output
    sample_char_dict = {0: ['sample origin: Primary', 'sample origin: Xenograft', 'sample origin: Ascites', 
                           'histology: Adenocarcinoma NOS', 'sample origin: ascites', 'sample origin: primary'], 
                       1: ['histology: Mucinous', 'histology: Clear cell', 'histology: Papillary serous', 
                          'histology: Endometrioid', 'histology: Mullerian NOS', 
                          'histology: Mixed Endometrioid and Pap. serous', 'histology: Dysgerminoma', 
                          'histology: Carcinosarcoma', 'medium: RPMI', 'medium: OCMI', 'histology: Adenocarcinoma NOS'], 
                       2: ['sample type: fresh', 'sample type: frozen', 'medium: OCMI', None, 'medium: DMEM/F12', 
                          "medium: McCoy's 5A", 'medium: MCDB105/M199', "medium: Ham's F12"], 
                       3: ['medium: OCMI', None]}
    
    # Create a mock clinical DataFrame with histology information
    # Assume each unique value represents one sample
    samples = []
    trait_values = []
    
    # Extract values from row 1 (trait_row)
    for value in sample_char_dict[trait_row]:
        if pd.isna(value):
            continue
        samples.append(f"Sample_{len(samples) + 1}")
        trait_values.append(value)
    
    # Create a DataFrame with samples as columns
    data = {samples[i]: [trait_values[i]] for i in range(len(samples))}
    clinical_data = pd.DataFrame(data, index=[trait_row])
    
    # Extract and process clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the processed clinical features
    preview = preview_df(clinical_features)
    print("Preview of clinical features:")
    print(preview)
    
    # Save the processed clinical data
    # Ensure output directory exists
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:

# Set up paths for input files
clinical_data_path = os.path.join(in_cohort_dir, "clinical_data.csv")
gene_data_path = os.path.join(in_cohort_dir, "gene_data.csv")

# Check if gene expression data is available
is_gene_available = os.path.exists(gene_data_path)

# Define conversion functions for trait, age, and gender data
def convert_trait(value):
    if pd.isna(value):
        return None
    
    if isinstance(value, str):
        value_lower = value.lower().strip()
        # Extract the value part if in "label: value" format
        if ':' in value_lower:
            value_lower = value_lower.split(':', 1)[1].strip()
        
        # For endometrioid cancer studies, look for relevant keywords
        if any(term in value_lower for term in ['cancer', 'tumor', 'malignant', 'carcinoma', 'endometrioid']):
            return 1
        elif any(term in value_lower for term in ['normal', 'control', 'healthy', 'non-cancer']):
            return 0
    return None

def convert_age(value):
    if pd.isna(value):
        return None
    
    if isinstance(value, str):
        value_lower = value.lower().strip()
        # Extract the value part if in "label: value" format
        if ':' in value_lower:
            value_lower = value_lower.split(':', 1)[1].strip()
        
        # Extract numeric age value
        import re
        matches = re.search(r'(\d+)(?:\s*years?)?', value_lower)
        if matches:
            try:
                age = int(matches.group(1))
                return age
            except ValueError:
                pass
    return None

def convert_gender(value):
    if pd.isna(value):
        return None
    
    if isinstance(value, str):
        value_lower = value.lower().strip()
        # Extract the value part if in "label: value" format
        if ':' in value_lower:
            value_lower = value_lower.split(':', 1)[1].strip()
        
        if any(term in value_lower for term in ['female', 'woman', 'women', 'f']):
            return 0
        elif any(term in value_lower for term in ['male', 'man', 'men', 'm']):
            return 1
    return None

# Initialize variables
trait_row = None
age_row = None
gender_row = None
is_trait_available = False

# Check if clinical data file exists and process it
if os.path.exists(clinical_data_path):
    clinical_data = pd.read_csv(clinical_data_path)
    
    print("Clinical data shape:", clinical_data.shape)
    print("Clinical data columns:", clinical_data.columns.tolist())
    
    # Examine the sample characteristics to find trait, age, and gender information
    sample_characteristics = {}
    for i in range(len(clinical_data)):
        row_values = clinical_data.iloc[i].dropna().tolist()
        unique_values = set(row_values)
        sample_characteristics[i] = list(unique_values)
        print(f"Row {i} unique values: {sample_characteristics[i]}")
    
    # Identify rows containing trait, age, and gender information
    for row_idx, values in sample_characteristics.items():
        for value in values:
            if isinstance(value, str):
                value_lower = value.lower()
                
                # Check for trait indicators (cancer/normal status)
                if any(term in value_lower for term in ['cancer', 'tumor', 'carcinoma', 'normal', 'control', 'endometrioid']):
                    trait_row = row_idx
                
                # Check for age indicators
                if 'age' in value_lower and any(char.isdigit() for char in value_lower):
                    age_row = row_idx
                
                # Check for gender indicators
                if any(term in value_lower for term in ['gender', 'sex', 'female', 'male']):
                    gender_row = row_idx
    
    print(f"Identified rows - Trait: {trait_row}, Age: {age_row}, Gender: {gender_row}")
    
    # Check if trait data is available
    is_trait_available = trait_row is not None
else:
    print("Clinical data file not found.")
    clinical_data = pd.DataFrame()  # Empty dataframe

# Validate and save initial cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# If trait data is available, extract and save clinical features
if is_trait_available:
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    # Preview the extracted clinical data
    preview = preview_df(selected_clinical_df)
    print("\nPreview of extracted clinical data:")
    print(preview)
    
    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save extracted clinical data
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to: {out_clinical_data_file}")
else:
    print("Trait data is not available.")


### Step 4: Gene Data Extraction

In [ ]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


### Step 5: Gene Identifier Review

In [ ]:
# The identifiers start with "ILMN_" which indicates these are Illumina microarray probe IDs
# These are not standard human gene symbols, but rather probe identifiers from the Illumina platform
# We need to map these probe IDs to standard gene symbols for proper analysis

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


### Step 7: Gene Identifier Mapping

In [ ]:
# Extract the gene annotation from the SOFT file using the helper function
gene_annotation = get_gene_annotation(soft_file)

# Identifying the relevant columns for mapping
# Based on the preview, we can see:
# - 'ID' contains the probe IDs (ILMN_*)
# - 'Symbol' contains gene symbols

# Create the gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'Symbol')

# Print the first few rows of the mapping to verify
print("Gene mapping preview (first 5 rows):")
print(gene_mapping.head())

# Apply the gene mapping to convert probe-level data to gene-level data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print the first few gene symbols and dimensions of the gene expression data
print("\nGene data converted to gene symbols:")
print(f"Shape: {gene_data.shape}")
print(f"First 10 gene symbols: {gene_data.index[:10].tolist()}")

# Check if the mapping was successful by counting non-null gene entries
num_genes = len(gene_data.index)
print(f"Number of genes after mapping: {num_genes}")

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to: {out_gene_data_file}")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Re-extract the gene expression data since it wasn't persisted from the previous step
gene_data = get_genetic_data(matrix_file)
print(f"Re-extracted gene data shape: {gene_data.shape}")

# 1. Normalize gene symbols in the gene expression data
# First create the gene mapping again
gene_annotation = get_gene_annotation(soft_file)
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'Symbol')
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Now normalize the mapped gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Properly extract the clinical data from the matrix file
background_info, clinical_raw = get_background_and_clinical_data(matrix_file)

# Define trait row based on the sample characteristics from Step 1
trait_row = 1  # Row containing histology information including "Endometrioid"

# Define the conversion functions
def convert_trait(value):
    """Convert histology data to binary for Endometrioid_Cancer"""
    if pd.isna(value):
        return None
    
    # Extract the value after the colon if present
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary based on histology
    if isinstance(value, str) and "Endometrioid" in value:
        return 1  # Presence of Endometrioid cancer
    else:
        return 0  # Other histology types

# Age and gender not available in this dataset
age_row = None
gender_row = None
convert_age = None
convert_gender = None

try:
    # Extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_raw,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    print("Clinical features:")
    print(clinical_features)
    
    # Save clinical features to file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")
    
    # 3. Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
    print(f"Linked data shape: {linked_data.shape}")
    print("Linked data preview (first 5 rows, first 5 columns):")
    print(linked_data.iloc[:5, :5] if not linked_data.empty else "Empty DataFrame")
    
    # 4. Handle missing values
    if not linked_data.empty:
        print("Missing values before handling:")
        print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
        if 'Age' in linked_data.columns:
            print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
        if 'Gender' in linked_data.columns:
            print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")
        
        gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
        print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
        print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")
        
        cleaned_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {cleaned_data.shape}")
        
        # 5. Evaluate bias in trait and demographic features
        trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
        
        # 6. Final validation and save
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=trait_biased, 
            df=cleaned_data,
            note="Dataset contains gene expression data with histology information, including endometrioid cancer samples."
        )
        
        # 7. Save if usable
        if is_usable and not cleaned_data.empty:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            cleaned_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Data was determined to be unusable or empty and was not saved")
    else:
        print("No linked data could be created - either clinical or gene data is missing.")
        validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=False, 
            is_biased=True, 
            df=pd.DataFrame(),
            note="Dataset contains gene expression data but clinical-genetic data linking failed."
        )
        
except Exception as e:
    print(f"Error in clinical data processing: {e}")
    import traceback
    traceback.print_exc()
    
    # Still save the cohort info even if processing failed
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False, 
        is_biased=True, 
        df=pd.DataFrame(),
        note=f"Error during clinical data processing: {str(e)}"
    )
    print("Data was determined to be unusable due to processing errors and was not saved")